<div style="display: inline-block;">
    <img src="images/nhsa_logo.png" alt="Image" style="text-align: left;">
</div>

# Parent Gauge Data Analysis Project
---
## Data Wrangling Script and Documentation

In this script, we will provide a step-by-step demonstration of how script is being cleaned. 

In [1]:
# Start with the necessary imports
import pandas as pd
import numpy as np
from tabulate import tabulate
from prettytable import PrettyTable
from rich.console import Console
from rich.table import Table

#uses old version of google trans: pip3 install googletrans==3.1.0a0
from googletrans import Translator

#used for filling in missing genders --DISCLAIMER: I recognize the sensitivities of this matter and understand 
#that this may not be totally accurate of the individual. This will only be used for statistical purposes.
import gender_guesser.detector as gender_guesser
from genderize import Genderize
import sexmachine.detector as sexmachine

In [2]:
#Load the Data into the dataframe
df = pd.read_excel('../data/INTVDATA.xlsx', sheet_name ='Main', engine ='openpyxl')
######MAKE SURE TO SPECIFY DATATYPE LATER ON######

#Copy existing dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

#read the new .csv file
df = pd.read_csv('../data/intv_data.csv')

/var/folders/vw/1sthm_ts1dj_1hqg1xnsz19w0000gn/T/ipykernel_66155/428991487.py:9: DtypeWarning: Columns (13,18,20,30,38,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/intv_data.csv')


---------

## Drop Duplicate Rows

A majority of the duplicate rows in the parent gauge dataset involve rows that are duplicates of the header row. We will use a random variable such as 'date' and remove rows that equal the name of the variable. Afterwards, we will drop additional duplicates, if any. 

In [3]:
#To make sure all duplicate "header" rows are eliminated, we pick a random column,
#remove rows where 'date' column is equal to 'date', except the first row
# Initialize a counter for deleted rows
deleted_date_rows = 0

# Iterate over the rows of the DataFrame
for i, row in df.iterrows():
    if row['date'] == 'date':
        # Remove the row if the date matches
        df.drop(i, inplace=True)
        deleted_date_rows += 1

# Count the number of rows before dropping duplicates
rows_before = len(df)

# Drop duplicate rows based on all columns except the first two, since they are indexed
result = df.drop_duplicates(subset=df.columns[2:])
        
# Count the number of rows after dropping duplicates
rows_after = len(result)

# Calculate and print the number of rows dropped
rows_dropped = (rows_before - rows_after) + deleted_date_rows
print(f"Number of duplicate rows dropped: {rows_dropped}")

#Copy existing dataframe to .csv file
df.to_csv('../data/intv_data.csv', index=False)

Number of duplicate rows dropped: 204


## Sample Data Frame Generation

Note: You can run this code again if you would like to reset the sample dataset.

In [4]:
#Because the main dataset is too large for data cleaning, 
#construct a small sample for faster processing. Once I am done coding, we will use the entire dataset.
df_sample = df.sample(frac=0.1)
df_sample.to_csv('../data/sample_data.csv', index=False)

print("Created a sample of 10% of the total dataset")

Created a sample of 10% of the total dataset


## Main Data Cleaning

This is a summary of all the data cleaning and reformatting steps that were conducted.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.
- **Program** - I identified the corresponding state and county.

# Code to Clean

This section examines each variable and transforms and cleans them accordingly.

## Center

For the center, we want to correspond each center to their respective state and geographic location for future analyses.

In [36]:
unique_centers = df_sample['center'].unique().tolist()

# Sort the list in place
unique_centers.sort()

#create a text file of the unique programs
with open('../data/unique_centers.txt', 'w') as f:
    for item in unique_centers:
        f.write("%s\n" % item)

## Created_at
 drop this feature, as it is unnecessary

## Date

Here, we will break down the date into three columns, seperating the year, month, and day. This code performs data cleaning on a column called 'date' in the dataset. It first checks each entry in the column for any errors in the date format, printing an error message if any are found. Then, it converts the 'date' column to a standard datetime format and creates new columns for the year, month, and day extracted from the dates. Finally, the cleaned data is saved to a CSV file. This code ensures that the dates are properly formatted, allows for easy analysis based on different time periods, and provides a clean dataset for further use.

In [47]:
# Iterate over the entries in the 'date' column
for i, date in enumerate(df_sample['date']):
    try:
        # Try to convert the date to datetime format
        pd.to_datetime(date, format='mixed')
    except Exception:
        print(f"An error occurred at index {i} with the date: {date}")
        
df_sample['date'] = pd.to_datetime(df_sample['date'], errors='coerce')

#Create separate 'year', 'month', and 'day' columns
df_sample['date_year'] = df_sample['date'].dt.year
df_sample['date_month'] = df_sample['date'].dt.month
df_sample['date_day'] = df_sample['date'].dt.day

#save to csv
df_sample.to_csv('../data/sample_data.csv', index=False)


## Evaluation

Dummy variables have been created, breaking the three categorical variables into three columns.

In [48]:
#use one-hot encoding to create dummy variables in preparation for regression.
#WARNING: This, however eliminates the original 'evaluation' column
df_sample = pd.get_dummies(df_sample, columns=['evaluation'])

#drop excess 'evaluation_evaluation' column, if it exists.
if 'evaluation_evaluation' in df_sample.columns:
    df_sample.drop('evaluation_evaluation', axis=1, inplace=True)

#save updates to csv
df_sample.to_csv('../data/sample_data.csv', index=False)


## Evaluation Year

The original format of the evaluation year was formatted as '2016-2017', for instance. For easier analysis, the start and end year have been split up into two columns, "evaluation_start_year" and "evaluation_end_year." Moreover, we convert the new variables into a faster data type. Lastly, try to fix any erroneous years, then remove the rest.

In [55]:
# Split the 'evaluation_year' column into two separate columns 'start_year' and 'end_year'
df_sample[['evaluation_start_year', 'evaluation_end_year']] = df_sample['evaluation_year'].str.split('-', expand=True)

# Convert 'evaluation_start_year' and 'evaluation_end_year to int16
df_sample['evaluation_start_year'] = df_sample['evaluation_start_year'].astype('int16')
df_sample['evaluation_end_year'] = df_sample['evaluation_end_year'].astype('int16')

# Fix errneous rows that were supposed to be a valid year
#####################

# Remove remaining erroneous rows where the 'evaluation_start_year' values are below 2015 or above 2023
df_sample = df_sample[(df_sample['evaluation_start_year'] >= 2015) & (df_sample['evaluation_start_year'] <= 2023)]

#save updates to csv
df_sample.to_csv('../data/sample_data.csv', index=False)

## Guardian Name

After employment the name should be scrubbed for privacy. 

## Guardian Employment

upon further inspection, it seems that the guardian employment is tied to the guardian_id, and it is not possible to further fill in missing values. note that over 60% of guardian_employment is missing. We should consider dropping this variable and removing it from our analysis.


## Guardian Enrollment Date

## Guardian Highest Education

same as guardian_employment, no referencing of the guardian_id will fill in missing blanks. Note that over 60% of guardian employment is misssing. We should consider dropping this variable and removing it from our analysis.

## Guardian, Hispanic?

There are missing values, but many other variables including the guardian's native language, student's native language, whether the student is hispanic, and what language the interview was used—to determine whether the guardian was hispanic.

In [61]:
##remove anything that is not "yes" or "no"
# Update the guardian_hispanic column
df_sample.loc[~df_sample["guardian_hispanic"].isin(["Yes", "No"]), "guardian_hispanic"] = ""

#check the column "guardian_native_language" "student_hispanic", 
#"student_native_language", "language of interview", 
# Define a function to fill missing values in guardian_hispanic column based on conditions
def fill_guardian_hispanic(row):
    if pd.isnull(row['guardian_hispanic']) or row['guardian_hispanic'] not in ['Yes', 'No']:
        if row['student_hispanic'] == 'Yes':
            return 'Yes'
        elif row['student_hispanic'] == 'No':
            return 'No'
        elif row['guardian_native_language'] == 'Spanish':
            return 'Yes'
        elif row['student_native_language'] == 'Spanish':
            return 'Yes'
        elif row['language'] == 'Spanish':
            return 'Yes'
        else:
            return 'No'
    else:
        return row['guardian_hispanic']

# Apply the function to fill missing values in guardian_hispanic column
df_sample['guardian_hispanic'] = df_sample.apply(fill_guardian_hispanic, axis=1)

# Convert "yes" and "no" to binary dummy variables
df_sample['guardian_hispanic'] = df_sample['guardian_hispanic'].map({'Yes': True, 'No': False})

# Write the updated data to a new CSV file
df_sample.to_csv('../data/sample_data.csv', index=False)

## Guardian Native Language

## Guardian Race

## Guardian Date of Birth

use guardian's DOB and interview year to determine guardian’s age during time of interview. We create a new variable named 'guardian_age'

In [66]:
# use guardian's DOB and interview year to determine guardian’s age during time of interview. 
# Convert 'guardian_birth_date' and 'date' columns to datetime
df_sample['guardian_birth_date'] = pd.to_datetime(df_sample['guardian_birth_date'], errors='coerce')
df_sample['date'] = pd.to_datetime(df_sample['date'], errors='coerce')

# Calculate the age at the time of the interview
df_sample['guardian_age'] = pd.NaT  # Initialize the column with missing values

for i, row in df_sample.iterrows():
    try:
        age = (row['date'] - row['guardian_birth_date']).days // 365
        df_sample.at[i, 'guardian_age'] = age
    except:
        pass  # Ignore any dates that are out of bounds

# Remove rows where 'guardian_age' values are less than 18 or above 99
df_sample = df_sample[(df_sample['guardian_age'] >= 18) & (df_sample['guardian_age'] <= 99)]

# Write the updated dataframe to a new CSV file
df_sample.to_csv('../data/sample_data.csv', index=False)

## Guardian Sex

Here, we use the gender guesser to fill in missing values. 

In [118]:
#clean some values for more proper analysis
df_sample['guardian_sex'] = df_sample['guardian_sex'].replace('F', 'Female')
df_sample['guardian_sex'] = df_sample['guardian_sex'].replace('M', 'Male')
df_sample['guardian_sex'] = df_sample['guardian_sex'].replace('unknown', '')

#save updates to csv
df_sample.to_csv('../data/sample_data.csv', index=False)

In [7]:
# use both the "gender_guesser", "Genderize", and "sexmachine" library to fill in missing values
# warning issue: running too many libraries in the function creates bottlenecks. 
def guess_gender(name):
    # Using gender_guesser library
    d_guesser = gender_guesser.Detector()
    gender_guess = d_guesser.get_gender(name)

    if gender_guess == 'male' or gender_guess == 'mostly_male':
        return 'Male'
    elif gender_guess == 'female' or gender_guess == 'mostly_female':
        return 'Female'

    # Using sexmachine library
    try:
        d_sexmachine = sexmachine.Detector(case_sensitive=False)
        genders_sexmachine = [d_sexmachine.get_gender(name) for name in names]

        for name, gender_sexmachine in zip(names, genders_sexmachine):
            if gender_sexmachine == 'male':
                results.append((name, 'Male'))
            elif gender_sexmachine == 'female':
                results.append((name, 'Female'))
    except:
        pass

    return None

# Apply guess_gender function to fill missing values in 'guardian_sex' column
df_sample['guardian_sex'] = df_sample.apply(lambda row: guess_gender(row['guardian']) if pd.isnull(row['guardian_sex']) else row['guardian_sex'], axis=1)
#df_sample['guardian_sex'] = df_sample.apply(lambda row: guess_gender(row['guardian']) if row['guardian_sex'] == '' else row['guardian_sex'], axis=1)
#df_sample['guardian_sex'] = df_sample.apply(lambda row: guess_gender(row['guardian']) if row['guardian_sex'] == 'unknown' else row['guardian_sex'], axis=1)

#save updates to csv
df_sample.to_csv('../data/sample_data.csv', index=False)

KeyboardInterrupt: 

In [114]:
table_breakdown = df_sample['guardian_sex'].value_counts(dropna=False).reset_index()
table_breakdown.columns = ['Value', 'Count']

print(table_breakdown)


     Value  Count
0   Female  18252
1     Male   1588
2      NaN    623
3    Other      3
4  Neutral      2


In [85]:
# Using direct mapping to create dummy variable out of guardian_sex
df_sample['female'] = np.where(df_sample['guardian_sex'] == 'Female', True, False)

#save updates to csv
df_sample.to_csv('../data/sample_data.csv', index=False)

## Guardian Vendor ID

consider dropping this feature

## Interview ID

drop this.

## Interviewer Name

Drop for privacy

## Interviewer ID

drop this.

## Interviewer Vendor ID

drop this.

## Language of Interview

Here, (1) remove invalid languages i.e. "language". (2) next, use the guardian ID to fill in missing values. (3) Lastly consider removing or imputing rows that have missing values. 

In [73]:
#step 1: remove invalid languages i.e. language

#step 2: use guardian_id to fill in missing values

#use guardian_native_language

#step 3: remove or impute rows that have missing values
#do this by checking if any of the open interview responses were in a certain language, if they are, apply corresponding language

## Mode of Interview

convert to dummy variable

In [5]:
#convert to dummy variable
# Using direct mapping to create dummy variable out of mode
df_sample['phone_interview'] = np.where(df_sample['mode'] == 'Phone', True, False)

#save updates to csv
df_sample.to_csv('../data/sample_data.csv', index=False)

## Program

## Student Name

drop the name for privacy.

In [77]:
##need to scrub the name, drop the column
#df_sample = df_sample.drop('student', axis=1)

## Student Enrollment Date

## Student Disability Status

*If entry is blank, change to 'None' if interview disregards questions intended for parents of students with disabilities. 

In [12]:
# Count the number of missing rows in 'guardian_hispanic' column
missing_count = df_sample['student_hispanic'].isnull().sum()

# Print the number of missing rows
print("Number of missing rows in 'student_hispanic' column:", missing_count)

Number of missing rows in 'student_hispanic' column: 1136


## Student, Hispanic
##remove anything that is not "yes" or "no"

In [13]:
##remove anything that is not "yes" or "no"
# Update the guardian_hispanic column
df_sample.loc[~df_sample["guardian_hispanic"].isin(["Yes", "No"]), "guardian_hispanic"] = ""

#check the column "guardian_native_language" "guardian_hispanic", "student_native_language"
# Define a function to fill missing values in student_hispanic column based on conditions
def fill_student_hispanic(row):
    if pd.isnull(row['student_hispanic']):
        if row['guardian_hispanic'] == 'Yes':
            return 'Yes'
        elif row['guardian_hispanic'] == 'No':
            return 'No'
        elif row['guardian_native_language'] == 'Spanish':
            return 'Yes'
        elif row['student_native_language'] == 'Spanish':
            return 'Yes'
        elif pd.notnull(row['student_native_language']):
            return 'No'
        else:
            return ''
    else:
        return row['student_hispanic']

# Apply the function to fill missing values in student_hispanic column
df_sample['student_hispanic'] = df_sample.apply(fill_student_hispanic, axis=1)

##---------------------------##

# Convert "yes" and "no" to binary dummy variables
#df_sample = pd.get_dummies(df_sample, columns=["guardian_hispanic"], prefix="guardian_hispanic", drop_first=True)

# Write the updated data to a new CSV file
df_sample.to_csv('../data/sample_data.csv', index=False)



## Student, ID

## Student Birth Date

In [9]:
# use student's DOB and interview year to determine student’s age during time of interview. 
# Convert 'student_birth_date' and 'date' columns to datetime
df_sample['student_birth_date'] = pd.to_datetime(df_sample['student_birth_date'], errors='coerce')
df_sample['date'] = pd.to_datetime(df_sample['date'], errors='coerce')

# Calculate the age at the time of the guardian's interview
df_sample['student_age'] = pd.NaT  # Initialize the column with missing values

for i, row in df_sample.iterrows():
    try:
        age = (row['date'] - row['student_birth_date']).days // 365
        df_sample.at[i, 'student_age'] = age
    except:
        pass  # Ignore any dates that are out of bounds

# Remove rows where 'student_age' values are less than -1 or above 6
df_sample = df_sample[(df_sample['student_age'] >= -1) & (df_sample['student_age'] <= 6)]

# Write the updated dataframe to a new CSV file
df_sample.to_csv('../data/sample_data.csv', index=False)

## Student in last year

## Student Native Language

## Student Program Type

## Student Race

## Student Service Type

## Student Sex

use gender_guesser again

In [91]:
def guess_gender(name):
    d = gender.Detector()
    gender_guess = d.get_gender(name)
    return gender_guess

# Apply guess_gender function to fill missing values in 'student_sex' column
df_sample['student_sex'] = df_sample.apply(lambda row: guess_gender(row['student']) if pd.isnull(row['student_sex']) else row['student_sex'], axis=1)



## Student Staff

drop

## Student Staff ID

drop this column

## Student Staff Vendor ID

consider removing

## Student Vendor ID

remove

## Student Was Early Headstart

## Student Was Head Start

In [90]:
## Once everything is done: drop unnecesary columns

# Next Section: Likert Scale Interview Questions

# Next Section: Open Interview Questions

perhaps we can use data analysis to see how sentiments change
https://www.surveypractice.org/article/25699-what-to-do-with-all-those-open-ended-responses-data-visualization-techniques-for-survey-researchers

In [18]:
from googletrans import Translator

# Assuming 'df' is your DataFrame
columns_to_translate = ['OQ1', 'OQ2', 'OQ3', 'OQ3a', 'OQ4', 'OQ5', 'OQ6', 'OQ7', 'OQ8', 'OQ9', 'OQ10']

# Create an instance of the Translator class
translator = Translator(service_urls=['translate.google.com'])

# Iterate over the columns to translate
for column in columns_to_translate:
    # Translate the non-null values in the column
    df_sample[column] = df_sample[column].apply(lambda x: translator.translate(x, dest='en').text if pd.notnull(x) else x)


Exception: Could not find TKK token for this request.
See https://github.com/ssut/py-googletrans/issues/234 for more details.

In [15]:
#first need to translate spanish-language to english
# Assuming 'df' is your DataFrame
columns_to_translate = ['OQ1', 'OQ2', 'OQ3', 'OQ3a', 'OQ4', 'OQ5', 'OQ6', 'OQ7', 'OQ8', 'OQ9', 'OQ10']

# Create a table to display translation information
table = Table(title="Translation Information")
table.add_column("Variable", justify="left", style="cyan")
table.add_column("Rows Translated", justify="right", style="green")
table.add_column("% Translated (Non-Missing)", justify="right", style="magenta")

# Iterate over the columns to translate
for column in columns_to_translate:
    translated_count = 0
    non_missing_count = df[column].notnull().sum()
    translator = Translator(service_urls=['translate.google.com'])  # Create an instance of the Translator class
    for i, value in enumerate(df[column]):
        if pd.notnull(value):
            # Detect if the value is in Spanish
            if translator.detect(value).lang == "es":
                # Translate the value from Spanish to English
                translation = translator.translate(value, src='es', dest='en')
                # Update the translated value in the DataFrame
                df.loc[i, column] = translation.text
                translated_count += 1
    # Calculate the percentage of translated non-missing rows
    percent_translated = translated_count / non_missing_count * 100 if non_missing_count > 0 else 0
    # Add a row to the table
    table.add_row(column, str(translated_count), f"{percent_translated:.2f}%")

# Display the table
console = Console()
console.print(table)



#scrub names

Exception: Could not find TKK token for this request.
See https://github.com/ssut/py-googletrans/issues/234 for more details.

## Remove Unnecessary Columns

After finishing data cleaning, remove excess columns.

In [ ]:
##REMOVE UNNECESSARY COLUMNS
# guardian_vendor_id, interview_id, interviewer_id, interviewer,
#interviewer_vendor_id, student_staff_vendor_id, student_vendor_id
#guardian, guardian_employment, guardian_highest_education

# List of columns to be removed
columns_to_remove = ['created_at', 'guardian', 'guardian_employment', 'guardian_highest_education', 'guardian_birth_date'
                     'guardian_vendor_id', 'interview_id', 'interviewer_id', 'mode',
                     'interviewer', 'interviewer_vendor_id', 'student', 'student_staff_vendor_id', 'student_birth_date'
                     'student_vendor_id', 'student_staff', 'student_staff_id']

# Removing the columns from the DataFrame
df_sample = df_sample.drop(columns=columns_to_remove)

#save updates to working csv
df_sample.to_csv('../data/sample_data.csv', index=False)